In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch
import json

In [2]:
es=Elasticsearch(hosts='http://localhost:9200/')
client_info=es.info()
print("Connected to Elastic-Search !")
pprint(client_info.body)

Connected to Elastic-Search !
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'tGYnSe7ISE2SFo0pu1RcHg',
 'name': '5d9dbd669d35',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2023-02-13T09:35:20.314882762Z',
             'build_flavor': 'default',
             'build_hash': '2d58d0f136141f03239816a4e360a8d17b6d8f29',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.4.2',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.6.2'}}


# 1. `Creating Index`

1. simple way: 
- In this method, the mappings which define the structure of documents within an index are infered automatically.

In [3]:
es.indices.delete(index="my_index", ignore_unavailable=True)
es.indices.create(index="my_index")

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

- `shards and replicas`

In [4]:
es.indices.delete(index="my_index", ignore_unavailable=True)
es.indices.create(index="my_index",
                  settings={
                      "index":{
                          "number_of_replicas":2,
                          "number_of_shards":3
                      }
                  })

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

-` Inserting documents`

1. document structure
document={
    'title':'this is test title',
    'content':'hey this is content'
    'created_on':'2025-02-14'
}

In [5]:
doc={
    'title':'this is test title',
    'content':'hey this is content',
    'created_on':'2025-02-14'
}
response=es.index(index="my_index", document=doc)

In [6]:
pprint(response.body)

{'_id': 'yd8MEpUBZI6kT9oae1L8',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_shards': {'failed': 0, 'successful': 1, 'total': 3},
 '_version': 1,
 'result': 'created'}


-   get `mapping` of the different field of the documents in the index

In [7]:
mapping= es.indices.get_mapping(index="my_index")
pprint(mapping["my_index"]["mappings"]["properties"], indent=2)

{ 'content': { 'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
               'type': 'text'},
  'created_on': {'type': 'date'},
  'title': { 'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
             'type': 'text'}}


In [8]:
pprint(mapping["my_index"], indent=2)

{ 'mappings': { 'properties': { 'content': { 'fields': { 'keyword': { 'ignore_above': 256,
                                                                      'type': 'keyword'}},
                                             'type': 'text'},
                                'created_on': {'type': 'date'},
                                'title': { 'fields': { 'keyword': { 'ignore_above': 256,
                                                                    'type': 'keyword'}},
                                           'type': 'text'}}}}


-   `manual Mapping` (schema inforcement)
- it should be done before adding any document to the index. 
- it can be done why passing the mapping to the mapping parameter in 
```python
indices.create(index="index_name", mapping=mapping)
```
- or after creating the document mapping can be inforced using 

```python
indices.put_mapping(index="index_name", body=mapping)
```

In [9]:
mapping={
    "properties": {
        "content": {
            "fields": {"keyword": {"ignore_above": 256, "type": "keyword"}},
            "type": "text",
        },
        "created_on": {"type": "date"},
        "title": {
            "fields": {"keyword": {"ignore_above": 256, "type": "keyword"}},
            "type": "text",
        },
    }
}

-   creating mapping at the time of index creation

In [10]:
es.indices.delete(index="my_index", ignore_unavailable=True)

ObjectApiResponse({'acknowledged': True})

In [11]:
es.indices.create(index="my_index", mappings=mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [12]:
es.indices.get_mapping(index="my_index")['my_index']["mappings"]

{'properties': {'content': {'type': 'text',
   'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
  'created_on': {'type': 'date'},
  'title': {'type': 'text',
   'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}}

- creating mapping after index creating but before adding document

In [13]:
es.index(index="my_index",document=doc)

ObjectApiResponse({'_index': 'my_index', '_id': 'yt8MEpUBZI6kT9oafVIH', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

In [14]:
mapping2={
    "properties": {
        "content_data": { #changing the field
            "fields": {"keyword": {"ignore_above": 256, "type": "keyword"}},
            "type": "text",
        },
        "created_on": {"type": "date"},
        "title": {
            "fields": {"keyword": {"ignore_above": 256, "type": "keyword"}},
            "type": "text",
        },
    }
}

In [15]:
es.indices.put_mapping(index="my_index", body=mapping2)

ObjectApiResponse({'acknowledged': True})

In [16]:
pprint(es.indices.get_mapping(index="my_index")["my_index"]["mappings"], indent=2)

{ 'properties': { 'content': { 'fields': { 'keyword': { 'ignore_above': 256,
                                                        'type': 'keyword'}},
                               'type': 'text'},
                  'content_data': { 'fields': { 'keyword': { 'ignore_above': 256,
                                                             'type': 'keyword'}},
                                    'type': 'text'},
                  'created_on': {'type': 'date'},
                  'title': { 'fields': { 'keyword': { 'ignore_above': 256,
                                                      'type': 'keyword'}},
                             'type': 'text'}}}


- this show that the schema is updated but the latest schema is not implemented completely... it is just being updated with new fileds

In [17]:
es.indices.delete(index="my_index", ignore_unavailable=True)
es.indices.create(index="my_index")

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [18]:
es.indices.put_mapping(index="my_index", body=mapping)

ObjectApiResponse({'acknowledged': True})

In [19]:
pprint(es.indices.get_mapping(index="my_index")["my_index"]["mappings"],indent=2)

{ 'properties': { 'content': { 'fields': { 'keyword': { 'ignore_above': 256,
                                                        'type': 'keyword'}},
                               'type': 'text'},
                  'created_on': {'type': 'date'},
                  'title': { 'fields': { 'keyword': { 'ignore_above': 256,
                                                      'type': 'keyword'}},
                             'type': 'text'}}}
